In [1]:
import os
os.environ["MKL_NUM_THREADS"] = "1" 
os.environ["NUMEXPR_NUM_THREADS"] = "1" 
os.environ["OMP_NUM_THREADS"] = "1" 

In [3]:
import xarray as xr
import numpy as np
import dask
import dask_ml
from dask_ml.cluster import KMeans
import dask.array as da
import dask.delayed
import sklearn
from sklearn.mixture import GaussianMixture 
import seaborn as sns
import matplotlib.pyplot as plt
import multiprocessing as mp
import time
import random

import preprocessing_speed

ContextualVersionConflict: (scikit-learn 0.22.2.post1 (/export/home1/agarciaj/anaconda3/lib/python3.7/site-packages), Requirement.parse('scikit-learn>=0.23'), {'dask-ml'})

In [3]:
# from dask.distributed import Client

# client = Client()

In [4]:
%load_ext line_profiler 

## Load data

## Preprocessing

In [5]:
file_path1 = '../datasets/global-reanalysis-phy-001-030-monthly_med_2018.nc'
file_path2 = '/home/ref-ocean-reanalysis/global-reanalysis-phy-001-030-daily/2018/01/mercatorglorys12v1_gl12_mean_20180101_R20180103.nc'
file_path3 = '/home/ref-ocean-reanalysis/global-reanalysis-phy-001-030-daily/2018/01/*3.nc'
file_path4 = '/home/ref-ocean-reanalysis/global-reanalysis-phy-001-030-daily/2018/01/*10.nc'
file_path5 = '/home/ref-ocean-reanalysis/global-reanalysis-phy-001-030-daily/2018/01/*.nc'
var_name_ds = 'thetao'

In [ ]:
%lprun -f preprocessing_speed.preprocessing_allin dask_X = preprocessing_speed.preprocessing_allin(path=file_path5, scaling=True, backend='dask', multiple=True, var_name=var_name_ds)

size full DS: 210.77493396773934 Go
size after selection of variable: 50.911838088184595 Go


# PCA testing

In [7]:
dask_X['thetao'].data

dask.array<transpose, shape=(168235543, 30), dtype=float32, chunksize=(5426953, 30), chunktype=numpy.ndarray>

## Dask

In [14]:
dask_train = dask_X['thetao'][:50_000_000]#.data.rechunk(500000)
dask_train

<xarray.DataArray 'thetao' (sample_dim: 50000000, depth: 30)>
dask.array<getitem, shape=(50000000, 30), dtype=float32, chunksize=(5426953, 30), chunktype=numpy.ndarray>
Coordinates:
  * depth       (depth) float32 -0.494025 -1.541375 ... -318.1274 -380.213
  * sample_dim  (sample_dim) MultiIndex
  - time        (sample_dim) datetime64[ns] 2018-01-01T12:00:00 ... 2018-01-1...
  - latitude    (sample_dim) float64 -76.92 -76.92 -76.92 ... -47.92 -47.92
  - longitude   (sample_dim) float64 -180.0 -179.9 -179.8 ... -24.08 -24.0
Attributes:
    long_name:      Temperature
    standard_name:  sea_water_potential_temperature
    units:          degrees_C
    unit_long:      Degrees Celsius
    valid_min:      -32724
    valid_max:      21099
    cell_methods:   area: mean

In [15]:
print("size train: " + str(dask_train.nbytes/1073741824) + ' Go')

size train: 5.587935447692871 Go


In [ ]:
%%time
from dask_ml.decomposition import PCA as dask_pca

d_pca = dask_pca(n_components = 3, svd_solver='full')
d_pca.fit(dask_train)

In [ ]:
%%time
d_pca.explained_variance_ratio_.sum()

In [48]:
# %%time
# from dask_ml.decomposition import IncrementalPCA as dask_inc_pca

# d_inc = dask_inc_pca(n_components=3, svd_solver='full')
# d_inc.fit(dask_train)

In [49]:
# d_inc.explained_variance_ratio_.sum()

## SKlearn

In [16]:
%%time
from sklearn.decomposition import PCA as sk_pca
s_pca = sk_pca(n_components = 3, svd_solver='full')
s_pca.fit(dask_train)

CPU times: user 2min 48s, sys: 51.1 s, total: 3min 39s
Wall time: 3min


PCA(n_components=3, svd_solver='full')

In [17]:
s_pca.explained_variance_ratio_.sum()

0.9977344594527435

In [ ]:
%%time
from sklearn.decomposition import IncrementalPCA as sk_inc_pca
sk_inc = sk_inc_pca(n_components = 3)
sk_inc.fit(dask_train)

In [ ]:
sk_inc.explained_variance_ratio_.sum()

# Clustering comparison between Kmean and GMM

In [8]:
%%time
sample_size = 5_000_000
x_train = []
choice = np.random.choice(x.dims['sample_dim'], sample_size)
x_train = x['thetao'][choice]
x_train

/home1/datahome/lbachelo/conda-env/OceanPatterns2/lib/python3.8/site-packages/xarray/core/indexing.py:1343: PerformanceWarning: Slicing with an out-of-order index is generating 612231 times more chunks
  return self.array[key]


CPU times: user 16.3 s, sys: 1.56 s, total: 17.9 s
Wall time: 17.9 s


<xarray.DataArray 'thetao' (sample_dim: 5000000, depth: 30)>
dask.array<getitem, shape=(5000000, 30), dtype=float32, chunksize=(8, 30), chunktype=numpy.ndarray>
Coordinates:
  * depth       (depth) float32 -0.494025 -1.541375 ... -318.1274 -380.213
  * sample_dim  (sample_dim) MultiIndex
  - time        (sample_dim) datetime64[ns] 2018-01-04T12:00:00 ... 2018-01-0...
  - latitude    (sample_dim) float64 -34.92 87.5 -37.92 ... -19.92 -25.92 -43.25
  - longitude   (sample_dim) float64 -27.08 82.25 -1.75 ... 178.6 6.25 79.58
Attributes:
    long_name:      Temperature
    standard_name:  sea_water_potential_temperature
    units:          degrees_C
    unit_long:      Degrees Celsius
    valid_min:      -32756
    valid_max:      21085
    cell_methods:   area: mean

In [ ]:
print("size of x_train after random selection: " + str(x_train.nbytes/1073741824) + ' Go')

In [ ]:
# Building the model
K = 6
sklearn_kmean = sklearn.cluster.KMeans(n_clusters= K, n_init=10, max_iter=1000)
dask_kmean = KMeans(n_clusters= K, init_max_iter=10, max_iter=1000, init='k-means||')
batch_kmean = sklearn.cluster.MiniBatchKMeans(n_clusters=K, n_init=10, max_iter=1000, batch_size=100)
# sklearn_GMM = GaussianMixture(n_components=K, max_iter=1000, tol=1e-6)

## fit

In [13]:
# sklearn kmean
exec_sklearn_Kmean = []
for i in range(0,3):
    start_time = time.time()
    sklearn_kmean.fit(x_train)
    tmp_time = time.time() - start_time
    exec_sklearn_Kmean.append(tmp_time)
    print(str(i) + " : " + str(tmp_time))

print("fastest: " + str(np.min(exec_sklearn_Kmean)))
print("mean: " + str(np.mean(exec_sklearn_Kmean)))
print("slowest: " + str(np.max(exec_sklearn_Kmean)))

0 : 175.9372594356537
1 : 193.25755143165588
2 : 193.5323691368103
fastest: 175.9372594356537
mean: 187.57572666803995
slowest: 193.5323691368103


In [14]:
# dask kmean
exec_dask_Kmean = []
for i in range(0,3):
    start_time = time.time()
    dask_kmean.fit(x_train)
    tmp_time = time.time() - start_time
    exec_dask_Kmean.append(tmp_time)
    print(str(i) + " : " + str(tmp_time))

print("fastest: " + str(np.min(exec_dask_Kmean)))
print("mean: " + str(np.mean(exec_dask_Kmean)))
print("slowest: " + str(np.max(exec_dask_Kmean)))

0 : 59.65338206291199
1 : 66.84845423698425
2 : 54.15013408660889
fastest: 54.15013408660889
mean: 60.21732346216837
slowest: 66.84845423698425


In [15]:
# sklearn kmean mini batch
exec_batch = []
for i in range(0,10):
    start_time = time.time()
    batch_kmean.fit(x_train)
    tmp_time = time.time() - start_time
    exec_batch.append(tmp_time)
    print(str(i) + " : " + str(tmp_time))

print("fastest: " + str(np.min(exec_batch)))
print("mean: " + str(np.mean(exec_batch)))
print("slowest: " + str(np.max(exec_batch)))

0 : 20.18517565727234
1 : 14.08262825012207
2 : 21.33794355392456
3 : 15.780104398727417
4 : 21.767130374908447
5 : 15.640953302383423
6 : 13.934991598129272
7 : 13.97908878326416
8 : 14.015222072601318
9 : 24.520397901535034
fastest: 13.934991598129272
mean: 17.524363589286803
slowest: 24.520397901535034


In [16]:
#sklearn GMM
# %time sklearn_GMM.fit(x_train)

## Pedict

In [31]:
#sklearn kmean
%time results = sklearn_kmean.predict(x['thetao'])

CPU times: user 2.95 s, sys: 6.22 s, total: 9.17 s
Wall time: 1.74 s


In [32]:
# dask kmean
%time results = dask_kmean.predict(x['thetao'])

CPU times: user 1.29 s, sys: 28 ms, total: 1.32 s
Wall time: 1.08 s


In [33]:
# sklearn GMM
# %time results = sklearn_GMM.predict(x['thetao'])

CPU times: user 49.3 s, sys: 25.9 s, total: 1min 15s
Wall time: 29.7 s


In [ ]:
# client.close()